<a id='home'></a>

### purpose

create table of locus counts for manuscript using printouts from `04_create_datasets-wza_baypass_random.ipynb`

### outline

1. [full data sets](#full)
    - how many loci were in the full wza and baypass sets (+ random/pseudo random)?
    
1. [RONA](#rona)

    - how many loci were used in RONA?

In [1]:
from pythonimports import *

latest_commit()
sinfo(html=True)

##################################################################
Current commit of pythonimports:
commit 2c7df50468a87779dfc99f033ba76da56aefb5a3  
Author: Brandon <brandon.lind@ubc.ca>  
Date:   Fri Dec 10 09:33:06 2021 -0700
Today:	December 10, 2021 - 19:09:58
python version: 3.8.5
##################################################################



<a id='full'></a>
# 1. full data sets

[top](#home)

In [5]:
text = '''fdc-baypass-real 17516
fdi-baypass-real 12398
jp-baypass-real 22635
combined-baypass-real 25219
fdc-wza-real 4886
fdi-wza-real 11434
jp-wza-real 4788
wl-wza-real 8496
combined-wza-real 14760
fdc-baypass-random 17516
fdi-baypass-random 12398
jp-baypass-random 22635
combined-baypass-random 25219
fdc-wza-random 4886
fdi-wza-random 11434
jp-wza-random 4788
wl-wza-random 8496
combined-wza-random 14760
fdc-baypass-pseudo_random_loci 17516
fdi-baypass-pseudo_random_loci 12398
jp-baypass-pseudo_random_loci 22635
combined-baypass-pseudo_random_loci 25219
fdc-wza-pseudo_random_loci 4886
fdi-wza-pseudo_random_loci 11434
jp-wza-pseudo_random_loci 4788
wl-wza-pseudo_random_loci 8496
combined-wza-pseudo_random_loci 14760'''

locus_counts = wrap_defaultdict(None, 3)
for line in text.split('\n'):
    dataset,count = line.split()
    spp,method,setname = dataset.split('-')
    locus_counts[spp][method][setname] = count

spp,method,setname,count

('combined', 'wza', 'pseudo_random_loci', '14760')

In [7]:
for spp,_dict in locus_counts.items():
    print(ColorText(spp).bold())
    display(pd.DataFrame(_dict))

fdc


,baypass,wza
real,17516,4886
random,17516,4886
pseudo_random_loci,17516,4886


fdi


,baypass,wza
real,12398,11434
random,12398,11434
pseudo_random_loci,12398,11434


jp


,baypass,wza
real,22635,4788
random,22635,4788
pseudo_random_loci,22635,4788


combined


,baypass,wza
real,25219,14760
random,25219,14760
pseudo_random_loci,25219,14760


wl


,wza
pseudo_random_loci,8496
random,8496
real,8496


<a id='rona'></a>
# 2. RONA

[top](#home)

In [39]:
# get files that have loci IDs among sets created in ../04_create_datasets-wza_baypass_random.ipynb#outliers
training_dir = '/data/projects/pool_seq/phenotypic_data/offset_misc_files/training/training_files'
files = fs(training_dir, 'full', endswith='.txt', exclude='envdata')
len(files)

27

In [40]:
# get the loci that were assigned to testing sets in ../04_create_datasets-wza_baypass_random.ipynb#outliers
    # ie union of random/baypass/wza/psuedo_random sets
spploci = defaultdict(list)
grouploci = wrap_defaultdict(None, 3)
for f in files:
    spp,method,setname,kfold = op.basename(f).rstrip('.txt').split('-')
    
    df = pd.read_table(f, index_col='index', nrows=1)
    # this next line from a previous run where I had accidentally wrote txt with multiple index 
        # (can prob deprecate, but no harm if I leave it)
    loci = [locus for locus in df.columns if 'Unnamed' not in locus and 'level' not in locus]
    
    print(spp, method, setname, len(loci))
    grouploci[spp][method][setname] = loci
    spploci[spp].extend(loci)
    
print('\n')

for spp,loci in spploci.items():
    spploci[spp] = uni(loci)
    print(spp, len(spploci[spp]))

combined baypass pseudo_random_loci 25219
combined baypass random 25219
combined baypass real 25219
combined wza pseudo_random_loci 14760
combined wza random 14760
combined wza real 14760
fdc baypass pseudo_random_loci 17516
fdc baypass random 17516
fdc baypass real 17516
fdc wza pseudo_random_loci 4886
fdc wza random 4886
fdc wza real 4886
fdi baypass pseudo_random_loci 12398
fdi baypass random 12398
fdi baypass real 12398
fdi wza pseudo_random_loci 11434
fdi wza random 11434
fdi wza real 11434
jp baypass pseudo_random_loci 22635
jp baypass random 22635
jp baypass real 22635
jp wza pseudo_random_loci 4788
jp wza random 4788
jp wza real 4788
wl wza pseudo_random_loci 8496
wl wza random 8496
wl wza real 8496


combined 114396
fdc 64714
fdi 68488
jp 76320
wl 25378


In [24]:
ronadir = '/data/projects/pool_seq/phenotypic_data/offset_misc_files/results/rona'

In [41]:
results = pklload(op.join(ronadir, 'linear_models.pkl'))

# determine which of the loci had pvals <= 0.05
keep = wrap_defaultdict(list, 2)
for spp,envdict in results.items():
    for env,locusdict in pbar(envdict.items(), desc=spp):
        for locus,(slope,intercept,pval) in locusdict.items():
            if pval <= 0.05:
                keep[spp][env].append(locus)


combined: 100%|██████████| 20/20 [00:00<00:00, 27.43it/s]


In [46]:
keep['jp']['Elevation'][0]

'>super4-514038'

In [49]:
keepers = {}
for spp,envdict in keep.items():
    keepers[spp] = uni(flatten(envdict.values()))
    print(spp, len(keepers[spp]))

jp 38991
fdi 36930
fdc 35340
combined 103853


In [58]:
rona_table = wrap_defaultdict(None, 2)
for (spp,method,setname),loci in unwrap_dictionary(grouploci):
    if spp != 'wl':
        interloci = set(loci).intersection(keepers[spp])
        print(spp, method, setname, len(loci), len(interloci))
        rona_table[spp][f'{method}-{setname}'] = len(interloci)

combined baypass pseudo_random_loci 25219 20761
combined baypass random 25219 23147
combined baypass real 25219 24844
combined wza pseudo_random_loci 14760 12023
combined wza random 14760 13537
combined wza real 14760 14629
fdc baypass pseudo_random_loci 17516 1571
fdc baypass random 17516 10034
fdc baypass real 17516 17436
fdc wza pseudo_random_loci 4886 433
fdc wza random 4886 2761
fdc wza real 4886 4881
fdi baypass pseudo_random_loci 12398 459
fdi baypass random 12398 7060
fdi baypass real 12398 12276
fdi wza pseudo_random_loci 11434 425
fdi wza random 11434 6583
fdi wza real 11434 11305
jp baypass pseudo_random_loci 22635 811
jp baypass random 22635 11899
jp baypass real 22635 22571
jp wza pseudo_random_loci 4788 201
jp wza random 4788 2538
jp wza real 4788 4788


In [67]:
for spp,_dict in rona_table.items():
    df = pd.DataFrame(_dict, index=[spp] * len(_dict))
    print(spp)
    display(df)


combined


,baypass-pseudo_random_loci,baypass-random,baypass-real,wza-pseudo_random_loci,wza-random,wza-real
combined,20761,23147,24844,12023,13537,14629
combined,20761,23147,24844,12023,13537,14629
combined,20761,23147,24844,12023,13537,14629
combined,20761,23147,24844,12023,13537,14629
combined,20761,23147,24844,12023,13537,14629
combined,20761,23147,24844,12023,13537,14629


fdc


,baypass-pseudo_random_loci,baypass-random,baypass-real,wza-pseudo_random_loci,wza-random,wza-real
fdc,1571,10034,17436,433,2761,4881
fdc,1571,10034,17436,433,2761,4881
fdc,1571,10034,17436,433,2761,4881
fdc,1571,10034,17436,433,2761,4881
fdc,1571,10034,17436,433,2761,4881
fdc,1571,10034,17436,433,2761,4881


fdi


,baypass-pseudo_random_loci,baypass-random,baypass-real,wza-pseudo_random_loci,wza-random,wza-real
fdi,459,7060,12276,425,6583,11305
fdi,459,7060,12276,425,6583,11305
fdi,459,7060,12276,425,6583,11305
fdi,459,7060,12276,425,6583,11305
fdi,459,7060,12276,425,6583,11305
fdi,459,7060,12276,425,6583,11305


jp


,baypass-pseudo_random_loci,baypass-random,baypass-real,wza-pseudo_random_loci,wza-random,wza-real
jp,811,11899,22571,201,2538,4788
jp,811,11899,22571,201,2538,4788
jp,811,11899,22571,201,2538,4788
jp,811,11899,22571,201,2538,4788
jp,811,11899,22571,201,2538,4788
jp,811,11899,22571,201,2538,4788
